In [1]:
import h5py
import numpy as np
import os, time
gpu = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn.utils.prune as prune
import torch.nn as nn

import torchvision

torch.manual_seed(0)
np.random.seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Machine has %d GPU'%torch.cuda.device_count())

Machine has 1 GPU


In [3]:
dataset_path = "/workspace/dataset/GOLD_XYZ_OSC.0001_1024.hdf5"
os.path.isfile(dataset_path)

True

In [4]:
# Prepare data loader
batch_size = 1024

class radioml_18_dataset(Dataset):
    def __init__(self, dataset_path):
        super(radioml_18_dataset, self).__init__()
        h5_file = h5py.File(dataset_path,'r')
        self.data = h5_file['X']
        print(self.data.shape)
        self.mod = np.argmax(h5_file['Y'], axis=1) # comes in one-hot encoding
        self.snr = h5_file['Z'][:,0]
        self.len = self.data.shape[0]

        self.mod_classes = ['OOK','4ASK','8ASK','BPSK','QPSK','8PSK','16PSK','32PSK',
        '16APSK','32APSK','64APSK','128APSK','16QAM','32QAM','64QAM','128QAM','256QAM',
        'AM-SSB-WC','AM-SSB-SC','AM-DSB-WC','AM-DSB-SC','FM','GMSK','OQPSK']
        self.snr_classes = np.arange(-20., 32., 2) # -20dB to 30dB

        # do not touch this seed to ensure the prescribed train/test split!
        np.random.seed(2018)
        train_indices = []
        test_indices = []
        for mod in range(0, 24): # all modulations (0 to 23)
            for snr_idx in range(0, 26): # all SNRs (0 to 25 = -20dB to +30dB)
                # 'X' holds frames strictly ordered by modulation and SNR
                start_idx = 26*4096*mod + 4096*snr_idx
                indices_subclass = list(range(start_idx, start_idx+4096))
                
                # 90%/10% training/test split, applied evenly for each mod-SNR pair
                split = int(np.ceil(0.1 * 4096)) 
                np.random.shuffle(indices_subclass)
                train_indices_subclass = indices_subclass[split:]
                test_indices_subclass = indices_subclass[:split]
                
                # you could train on a subset of the data, e.g. based on the SNR
                # here we use all available training samples
                if snr_idx >= 0:
                    train_indices.extend(train_indices_subclass)
                test_indices.extend(test_indices_subclass)
                
        self.train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
        self.test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)

    def __getitem__(self, idx):
        # transpose frame into Pytorch channels-first format (NCL = -1,2,1024)
        return self.data[idx].transpose(), self.mod[idx], self.snr[idx]

    def __len__(self):
        return self.len


dataset = radioml_18_dataset(dataset_path)
train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=dataset.train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=dataset.test_sampler)

(2555904, 1024, 2)


In [5]:
import brevitas.nn as qnn
from brevitas.quant import IntBias
from brevitas.inject.enum import ScalingImplType
from brevitas.inject.defaults import Int8ActPerTensorFloatMinMaxInit


class InputQuantizer(Int8ActPerTensorFloatMinMaxInit):
    bit_width = 8
    min_val = -2.0
    max_val = 2.0
    scaling_impl_type = ScalingImplType.CONST # Fix the quantization range to [min_val, max_val]

class IlliNet(nn.Module):
    def __init__(self):
        super().__init__()
        a_bits = 4
        w_bits = 4
        self.input = qnn.QuantHardTanh(act_quant=InputQuantizer)
        
        self.conv0 = qnn.QuantConv1d(2, 16, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn0 = nn.BatchNorm1d(16)

        self.conv1 = qnn.QuantConv1d(16, 32, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = qnn.QuantConv1d(32, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = qnn.QuantConv1d(64, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn3 = nn.BatchNorm1d(64)

        self.conv4 = qnn.QuantConv1d(64, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn4 = nn.BatchNorm1d(64)

        self.conv5 = qnn.QuantConv1d(64, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn5 = nn.BatchNorm1d(64)

        self.conv6 = qnn.QuantConv1d(64, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn6 = nn.BatchNorm1d(64)

        self.conv7 = qnn.QuantConv1d(64, 64, 3, padding=1, weight_bit_width=w_bits, bias=False)
        self.bn7 = nn.BatchNorm1d(64)

        self.fc1 = qnn.QuantLinear(256, 64, weight_bit_width=w_bits, bias=False)
        self.bn_dense = nn.BatchNorm1d(64)
        self.fc2 = qnn.QuantLinear(64, 24, weight_bit_width=w_bits, bias=False)

        self.relu = qnn.QuantReLU(bit_width=a_bits)
        self.pooling = nn.MaxPool1d(2)

    def forward(self, x):
        x = self.input(x)

        x = self.conv0(x)
        x = self.bn0(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu(x)
        x = self.pooling(x)
        
        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu(x)
        x = self.pooling(x)

        x = x.view(x.size(0), -1)
        
        x = self.relu(self.bn_dense(self.fc1(x)))
        x = self.fc2(x)
        return x

net = IlliNet()

if torch.cuda.device_count() > 1:
    print('Muti-GPU activated')
    net = nn.DataParallel(net)

save_path = './IlliNet_trained.pth'
if os.path.isfile(save_path):
    saved_state = torch.load(save_path)
    net.load_state_dict(saved_state, strict=True)

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# First evaluate accuracy of the existing model
total = 0
correct = 0
global_accu = []
for i, data in enumerate(data_loader_test, 0):
    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = net(inputs)
    labels = labels.type(torch.long)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
global_accu.append(100 * correct / total)
# print('\n[Epoch %d] loss: %f'%(epoch + 1, running_loss))
print('Accuracy of the network: %f %%'%(100*correct/total))

In [7]:
def global_sparsity_eval():
      return 100. * float(torch.sum(net.conv0.weight == 0) 
                          + torch.sum(net.conv1.weight == 0) 
                          + torch.sum(net.conv2.weight == 0) 
                          + torch.sum(net.conv3.weight == 0) 
                          + torch.sum(net.conv4.weight == 0)
                          + torch.sum(net.conv5.weight == 0) 
                          + torch.sum(net.conv6.weight == 0) 
                          + torch.sum(net.conv7.weight == 0) 
                          + torch.sum(net.fc1.weight == 0) 
                          + torch.sum(net.fc2.weight == 0)) / float(net.conv0.weight.nelement() 
                            + net.conv1.weight.nelement() 
                            + net.conv2.weight.nelement() 
                            + net.conv3.weight.nelement() 
                            + net.conv4.weight.nelement() 
                            + net.conv5.weight.nelement() 
                            + net.conv6.weight.nelement() 
                            + net.conv7.weight.nelement()
                            + net.fc1.weight.nelement() 
                            + net.fc2.weight.nelement())
def layer_sparsity_eval():
    print('Layer weight sparsity values: ')
    print('Conv 0:', torch.sum(net.conv0.weight == 0).item()/net.conv0.weight.nelement())
    print('Conv 1:', torch.sum(net.conv1.weight == 0).item()/net.conv1.weight.nelement())
    print('Conv 2:', torch.sum(net.conv2.weight == 0).item()/net.conv2.weight.nelement())
    print('Conv 3:', torch.sum(net.conv3.weight == 0).item()/net.conv3.weight.nelement())
    print('Conv 4:', torch.sum(net.conv4.weight == 0).item()/net.conv4.weight.nelement())
    print('Conv 5:', torch.sum(net.conv5.weight == 0).item()/net.conv5.weight.nelement())
    print('Conv 6:', torch.sum(net.conv6.weight == 0).item()/net.conv6.weight.nelement())
    print('Conv 7:', torch.sum(net.conv7.weight == 0).item()/net.conv7.weight.nelement())
    print('FC 1:', torch.sum(net.fc1.weight == 0).item()/net.fc1.weight.nelement())
    print('FC 2:', torch.sum(net.fc2.weight == 0).item()/net.fc2.weight.nelement())

init_global_sparsity = global_sparsity_eval()
print(init_global_sparsity)
layer_sparsity_eval()

55.849476680132206
Layer weight sparsity values: 
Conv 0: 0.25
Conv 1: 0.3821614583333333
Conv 2: 0.4210611979166667
Conv 3: 0.5135904947916666
Conv 4: 0.593505859375
Conv 5: 0.53271484375
Conv 6: 0.536865234375
Conv 7: 0.5533040364583334
FC 1: 0.6407470703125
FC 2: 0.9264322916666666


In [8]:
from datetime import datetime
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_str = now.strftime("%y%m%d_%H%M")
print("date and time =", dt_str)

# Adjustable hyperparameters
model_name = 'IlliNet'

model_save_path = 'saves/%s_%s.pth'%(model_name, dt_str)
model_save_path_best = 'saves/%s_%s_best.pth'%(model_name, dt_str)
model_save_path_final = 'saves/%s_%s_final.pth'%(model_name, dt_str)

now = 2021-10-09 22:17:18.028140
date and time = 211009_2217


In [9]:
def test(model, test_loader):    
    model.eval() 
    y_true = []
    y_pred = []
    
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
        
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            labels = labels.type(torch.long)
            
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return 100*correct/total

In [10]:
global_loss = []
global_accu = []
num_epochs = 3

step = 0.015
sparsity = 0.0
spars_eval = 0.0

parameters_fc_only = (
    (net.fc1, 'weight'),
    (net.fc2, 'weight'),
)

parameters_all = (
    (net.conv0, 'weight'),
    (net.conv1, 'weight'),
    (net.conv2, 'weight'),
    (net.conv3, 'weight'),
    (net.conv4, 'weight'),
    (net.conv5, 'weight'),
    (net.conv6, 'weight'),
    (net.conv7, 'weight'),
    (net.fc1, 'weight'),
    (net.fc2, 'weight'),
)


best_acc = 0
ts = time.time()
net.to(device)

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    num_steps = len(train_dataloader)
    
    t2 = time.time()

    if epoch % 5 == 0:
        if epoch == 0:
            sparsity = global_sparsity_eval()/100
            print('Initialized model sparsity: %f %%'%(100*sparsity))
        else:
            sparsity = step
        prune.global_unstructured(parameters_all, pruning_method = prune.L1Unstructured, amount=sparsity)
    else:
        sparsity = step
        prune.global_unstructured(parameters_fc_only, pruning_method = prune.L1Unstructured, amount=sparsity)
        
    spars_eval = global_sparsity_eval()
    print('Sparsity after pruning: %f %%'%(spars_eval))
    
    for i, data in enumerate(train_dataloader, 0):
        t0 = time.time()
        
        inputs, labels = data[0].to(device), data[1].to(device)
            
        outputs = net(inputs)
        labels = labels.type(torch.long)
        
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        t1 = time.time()
        if i%1 == 0 or i == num_steps:
            print('\rStep %d/%d | Loss = %.4f, Time = %.6f'%(i+1, num_steps, loss, t1-t0), end='')
        
    training_acc = 100 * correct / total
    global_loss.append(running_loss)
    global_accu.append(training_acc)
    print('\n[Epoch %d] loss: %f'%(epoch + 1, running_loss))
    print('Training accuracy of the network: %f %%'%(training_acc))
    
    step = 0.015
    testing_acc = test(net, test_dataloader)
    print('Testing accuracy of the network: %f %%'%(testing_acc))
    
    if testing_acc < 57:
        step = 0
        print("Locking this epoch to increase accuracy")
        
    if testing_acc > best_acc and epoch != 0:
        print("Best model saved at ", model_save_path_best)
        torch.save(net.state_dict(), model_save_path_best)
        best_acc = testing_acc
    else:
        print("Model saved at ", model_save_path)
        torch.save(net.state_dict(), model_save_path)
        
    print('After epoch sparsity: %f %%'%(global_sparsity_eval()))
    layer_sparsity_eval()
    t3 = time.time()
    print('Epoch Time: %.4f '%(t3 - t2))
    print()
tt = time.time()
print('Finished Training, elapsed %.3f seconds'%(tt-ts))

Initialized model sparsity: 55.849477 %
Sparsity after pruning: 55.849477 %
Step 2247/2247 | Loss = 1.3795, Time = 0.049525
[Epoch 1] loss: 2811.759611
Training accuracy of the network: 56.821680 %
Testing accuracy of the network: 57.132583 %
Model saved at  saves/IlliNet_211009_2217.pth
After epoch sparsity: 55.849477 %
Layer weight sparsity values: 
Conv 0: 0.25
Conv 1: 0.3821614583333333
Conv 2: 0.4210611979166667
Conv 3: 0.5135904947916666
Conv 4: 0.593505859375
Conv 5: 0.53271484375
Conv 6: 0.536865234375
Conv 7: 0.5533040364583334
FC 1: 0.6407470703125
FC 2: 0.9264322916666666
Epoch Time: 635.5432 

Sparsity after pruning: 55.952763 %
Step 2247/2247 | Loss = 1.1959, Time = 0.030650
[Epoch 2] loss: 2823.689125
Training accuracy of the network: 56.611773 %
Testing accuracy of the network: 56.603737 %
Locking this epoch to increase accuracy
Best model saved at  saves/IlliNet_211009_2217_best.pth
After epoch sparsity: 55.952763 %
Layer weight sparsity values: 
Conv 0: 0.25
Conv 1: 0.

In [11]:
prune.remove(net.conv0, 'weight')
prune.remove(net.conv1, 'weight')
prune.remove(net.conv2, 'weight')
prune.remove(net.conv3, 'weight')
prune.remove(net.conv4, 'weight')
prune.remove(net.conv5, 'weight')
prune.remove(net.conv6, 'weight')
prune.remove(net.conv7, 'weight')
prune.remove(net.fc1, 'weight')
prune.remove(net.fc2, 'weight')

QuantLinear(
  in_features=64, out_features=24, bias=False
  (input_quant): ActQuantProxyFromInjector(
    (_zero_hw_sentinel): StatelessBuffer()
  )
  (output_quant): ActQuantProxyFromInjector(
    (_zero_hw_sentinel): StatelessBuffer()
  )
  (weight_quant): WeightQuantProxyFromInjector(
    (_zero_hw_sentinel): StatelessBuffer()
    (tensor_quant): RescalingIntQuant(
      (int_quant): IntQuant(
        (float_to_int_impl): RoundSte()
        (tensor_clamp_impl): TensorClampSte()
        (delay_wrapper): DelayWrapper(
          (delay_impl): _NoDelay()
        )
      )
      (scaling_impl): StatsFromParameterScaling(
        (parameter_list_stats): _ParameterListStats(
          (first_tracked_param): _ViewParameterWrapper(
            (view_shape_impl): OverTensorView()
          )
          (stats): _Stats(
            (stats_impl): AbsMax()
          )
        )
        (stats_scaling_impl): _StatsScaling(
          (affine_rescaling): Identity()
          (restrict_clamp_scaling

In [12]:
torch.save(net.state_dict(), model_save_path_final)